In [1]:
import math

## Put Options

### 1 step

In [4]:
S0 = 50.0     # current stock price
K  = 70.0     # strike price (deep ITM put)
sigma = 0.2   # volatility 
Rf = 0.10     # 10% risk-free rate for 1 year
R  = 1 + Rf   # gross risk-free return = 1.10
T  = 1.0          # total time to maturity in years
N  = 1            # number of steps
dt = T / N        # length of each step = 1.0 / 1 = 1 year

# find downard and upward movement (U and D) 
U = math.exp(sigma * math.sqrt(dt)) 
D = 1/U

# Risk-neutral probability (from binomial model) on up move
prob_U = (R - D) / (U - D) 

# Risk-neutral probability on a down move
prob_D = 1 - prob_U 


# Stock prices at maturity (up and down)
Su = S0 * U 
Sd = S0 * D


# I. Payoffs at maturity (t = 1 year) for 1-step tree, find P for T = 1
# calculate payoff of the option at maturity 
# formula for payoff at expiration for put option
Pd = max(K - Sd, 0.0)   
Pu = max(K - Su, 0.0) 



# II. Continuation value (European PUT value)
# Discount expected payoff using risk-neutral probabilities
continuation_value = ((prob_U * Pu) + (prob_D * Pd)) / R


# III. Intrinsic value today (if we exercised immediately)
intrinsic_today = max(K - S0, 0.0) 


# IV. European vs American prices
P0_european_1step = continuation_value                # European: can ONLY exercise at maturity
P0_american_1step = max(intrinsic_today, continuation_value)  # American: can exercise NOW or later


print("Risk-neutral probability Up Move:", prob_U)
print("Risk-neutral probability Down Move:", prob_D)
print("Stock price Up Move Su:", Su)
print("Stock price Down Move Sd:", Sd)
print("Put payoff at Su (Pu):", Pu)
print("Put payoff at Sd (Pd):", Pd)
print("Continuation value (European premium):", continuation_value)
print("Intrinsic value today:", intrinsic_today)
print("European put premium P0 (1 step):", P0_european_1step)
print("American put premium P0 (1 step):", P0_american_1step)

Risk-neutral probability Up Move: 0.6985070811282481
Risk-neutral probability Down Move: 0.3014929188717519
Stock price Up Move Su: 61.07013790800849
Stock price Down Move Sd: 40.936537653899094
Put payoff at Su (Pu): 8.92986209199151
Put payoff at Sd (Pd): 29.063462346100906
Continuation value (European premium): 13.636363636363631
Intrinsic value today: 20.0
European put premium P0 (1 step): 13.636363636363631
American put premium P0 (1 step): 20.0


### 2 steps

In [6]:
## 2 step
# Parameters
S0 = 50.0     # current stock price
K  = 70.0     # strike price (deep ITM put)
sigma = 0.2   # volatility 
Rf = 0.10     # 10% annual risk-free rate
T  = 1.0      # total time to maturity in years
N  = 2        # number of steps
dt = T / N    # length of each step = 0.5 years

# Per-step risk-free growth factor (simple annual rate scaled by dt)
R = 1 + Rf * dt    # = 1 + 0.10 * 0.5 = 1.05

# Up and down factors from volatility
U = math.exp(sigma * math.sqrt(dt))
D = 1 / U

# Risk-neutral probabilities (per step)
prob_U = (R - D) / (U - D)
prob_D = 1 - prob_U

# ---------- Stock prices at each node ----------
# t = 0
S_0  = S0

# t = dt
S_u  = S0 * U
S_d  = S0 * D

# t = T = 2*dt
S_uu = S0 * (U**2)
S_ud = S0 * U * D     # same as S_du in a recombining tree
S_dd = S0 * (D**2)

# I. Payoffs at maturity (T)
# Put payoff = max(K - S, 0)
P_uu = max(K - S_uu, 0.0)
P_ud = max(K - S_ud, 0.0)
P_dd = max(K - S_dd, 0.0)

# II. Backward induction: European put
# At t = dt (continuation values, no early exercise for European)
P_u_cont = (prob_U * P_uu + prob_D * P_ud) / R
P_d_cont = (prob_U * P_ud + prob_D * P_dd) / R

# At t = 0
P0_european_2step = (prob_U * P_u_cont + prob_D * P_d_cont) / R

# III. American put (allow early exercise)
# At t = dt: compare continuation vs intrinsic
intr_u = max(K - S_u, 0.0)
intr_d = max(K - S_d, 0.0)

P_u_amer = max(P_u_cont, intr_u)
P_d_amer = max(P_d_cont, intr_d)

# At t = 0: compare continuation vs intrinsic at t=0
intrinsic_today = max(K - S0, 0.0)
P0_american_2step = max(
    (prob_U * P_u_amer + prob_D * P_d_amer) / R,
    intrinsic_today
)

#  Print results
print("Risk-neutral probability Up Move:", prob_U)
print("Risk-neutral probability Down Move:", prob_D)
print("Stock price Up Move Su:", Su)
print("Stock price Down Move Sd:", Sd)
print("Put payoff at Su (Pu):", Pu)
print("Put payoff at Sd (Pd):", Pd)
print("Continuation value (European premium):", continuation_value)
print("Intrinsic value today:", intrinsic_today)
print("European put premium P0 (2-step):", P0_european_2step)
print("American  put premium P0 (2-step):", P0_american_2step)

Risk-neutral probability Up Move: 0.6408922805745831
Risk-neutral probability Down Move: 0.35910771942541686
Stock price Up Move Su: 61.07013790800849
Stock price Down Move Sd: 40.936537653899094
Put payoff at Su (Pu): 8.92986209199151
Put payoff at Sd (Pd): 29.063462346100906
Continuation value (European premium): 13.636363636363631
Intrinsic value today: 20.0
European put premium P0 (2-step): 13.492063492063485
American  put premium P0 (2-step): 20.0


#### N steps Model

In [8]:
S0 = 50.0
K  = 70.0
sigma = 0.5
Rf = 0.10
T  = 1.0
N  = 10                    
dt = T / N

# Per-step risk-free growth factor (simple annual rate scaled by dt)
R = 1 + Rf * dt               # simple compounding per step

# Up and down factors from volatility
U = math.exp(sigma * math.sqrt(dt))
D = 1 / U

# Risk-neutral probabilities (per step)
prob_U = (R - D) / (U - D)
prob_D = 1 - prob_U


# Loop Logic
# I. Payoffs at maturity (T) 
P = []        # option values at terminal nodes
for i in range(N + 1):
    # price = S0 * U^(number of up moves) * D^(number of down moves)
    ST = S0 * (U**(N-i)) * (D**i)
    P.append(max(K - ST, 0.0))      # PUT payoff


# II. Backward induction
# American & European trees calculated together
for step in range(N, 0, -1):
    newP = []
    for i in range(step):
        continuation = (prob_U * P[i] + prob_D * P[i+1]) / R
        european_val = continuation
        american_val = max(K - (S0 * (U**(step-1-i)) * (D**i)), continuation)

        newP.append(american_val)

    P = newP[:]    # replace layer

P0_american_50step = P[0]


# III. European value calculated separately (without early exercise) 
# rebuild payoff structure & do backward induction again
P_eur = []
for i in range(N + 1):
    ST = S0 * (U**(N-i)) * (D**i)
    P_eur.append(max(K - ST, 0.0))

for step in range(N, 0, -1):
    P_eur = [ (prob_U * P_eur[i] + prob_D * P_eur[i+1]) / R
              for i in range(step) ]

P0_european_50step = P_eur[0]

# print results
print(f"European Put ({N} step) C0:", P0_european_50step)
print(f"American Put ({N} step) C0:", P0_american_50step)



European Put (10 step) C0: 18.86542779860799
American Put (10 step) C0: 21.38970093921782


## Put - Loop Logic

In [10]:
## N periods - 1, 2, 10, 50, 100
## sigma - 0.2, 0.5, 1.0
def binomial_put(S0, K, sigma, Rf, T, N):
    dt = T / N
    R = 1 + Rf * dt
    U = math.exp(sigma * math.sqrt(dt))
    D = 1 / U

    prob_U = (R - D) / (U - D)
    prob_D = 1 - prob_U

    # payoff at maturity
    P_amer = []
    for i in range(N + 1):
        ST = S0 * (U**(N-i)) * (D**i)
        P_amer.append(max(K - ST, 0))

    P_eur = P_amer.copy()

    # backward induction
    for step in range(N, 0, -1):
        # European: continuation only
        P_eur = [ (prob_U * P_eur[i] + prob_D * P_eur[i+1]) / R
                  for i in range(step) ]

        # American: max(intrinsic, continuation)
        newP = []
        for i in range(step):
            continuation = (prob_U * P_amer[i] + prob_D * P_amer[i+1]) / R
            ST = S0 * (U**(step-1-i)) * (D**i)
            intrinsic = max(K - ST, 0)
            newP.append(max(intrinsic, continuation))
        P_amer = newP

    return P_eur[0], P_amer[0]


In [11]:
## N periods - 1, 2, 10, 50, 100
## sigma - 0.2
S0 = 50.0
K  = 70.0
sigma = 0.2        
Rf = 0.10
T  = 1.0

steps_list = [1, 2, 10, 50, 100]

for N in steps_list:
    european, american = binomial_put(S0, K, sigma, Rf, T, N)
    print(f"{N}-step European Put: {european:.6f}")
    print(f"{N}-step American Put:  {american:.6f}")
    print()

1-step European Put: 13.636364
1-step American Put:  20.000000

2-step European Put: 13.492063
2-step American Put:  20.000000

10-step European Put: 13.973636
10-step American Put:  20.000000

50-step European Put: 13.970770
50-step American Put:  20.000000

100-step European Put: 13.989193
100-step American Put:  20.000000



In [12]:
## N periods - 1, 2, 10, 50, 100
## sigma - 0.5
S0 = 50.0
K  = 70.0
sigma = 0.5        
Rf = 0.10
T  = 1.0

steps_list = [1, 2, 10, 50, 100]

for N in steps_list:
    european, american = binomial_put(S0, K, sigma, Rf, T, N)
    print(f"{N}-step European Put: {european:.6f}")
    print(f"{N}-step American Put:  {american:.6f}")
    print()

1-step European Put: 18.989438
1-step American Put:  20.000000

2-step European Put: 20.103984
2-step American Put:  21.749128

10-step European Put: 18.865428
10-step American Put:  21.389701

50-step European Put: 19.084901
50-step American Put:  21.445153

100-step European Put: 19.068076
100-step American Put:  21.436793



In [13]:
## N periods - 1, 2, 10, 50, 100
## sigma - 1
S0 = 50.0
K  = 70.0
sigma = 1       
Rf = 0.10
T  = 1.0

steps_list = [1, 2, 10, 50, 100]

for N in steps_list:
    european, american = binomial_put(S0, K, sigma, Rf, T, N)
    print(f"{N}-step European Put: {european:.6f}")
    print(f"{N}-step American Put:  {american:.6f}")
    print()

1-step European Put: 32.301277
1-step American Put:  32.301277

2-step European Put: 29.689325
2-step American Put:  31.712148

10-step European Put: 29.358003
10-step American Put:  30.883003

50-step European Put: 28.932014
50-step American Put:  30.566668

100-step European Put: 28.952337
100-step American Put:  30.563054



## Call Options

### 1 step

In [16]:
# 1-step example (to match the slide)
# Parameters from the slide
S0 = 50.0     # currrent stock price
K = 45.0      # strike price
U = 1.25      # estimate the size of an upmove is 25%
D = 0.80      # estimate the size of downmove, reciprocal of U, D = 1/U
Rf = 0.07     # 7% risk-free rate for 1 year
R = 1 + Rf    # gross risk-free return

# Risk-neutral probability (from binomial model) on up move
prob_U = (R - D) / (U - D)  #0.6

# Risk-neutral probability on a down move
prob_D = 1 - prob_U        # 0.4

# Stock prices at maturity (up and down) 
Su = S0 * U # 50 * 1.25 = 62.5
Sd = S0 * D # 50 * 0.8 = 40

# I. calculate Payoffs at maturity (t = 1 year) for 1-step tree
# calculate payoff of the option at maturity 
# formula for payoff at expiration for call option
Cu = max(Su - K, 0.0)   # 17,  up-state payoff
Cd = max(Sd - K, 0.0)   # 0, option expires out of money,  down-state payoff

# II. Continuation value  (European Value)
# calculate option value one year from now using psuedo probabilties
# discount new option value back to it's old option value using risk free rate
continuation_value = ((prob_U * Cu) + (prob_D * Cd)) / R   #(0.6* 17) + (0.4 *0) / 1 + 0.07 ~ 9.81

# III. Intrinsic value today(if we exercised immediately)
intrinsic_today = max(S0 - K, 0.0) # 50 -45 = 5.0

# IV. European vs American prices
P0_european_1step = continuation_value    # European: can ONLY exercise at maturity
C0_american_1step = max(intrinsic_today, continuation_value)  #American: can exercise NOW or later

print("Risk-neutral probability Up Move:", prob_U)
print("Risk-neutral probability Down Move:", prob_D)
print("Stock prices at maturity Up Move:", Su)
print("Payoff Up Move", Cu)
print("Stock prices at maturity Down Move::", Sd)
print("Payoff Up Move:", Cd)
print("Continuation value (European premium):", continuation_value)
print("Intrinsic value today:", intrinsic_today)
print("European put premium P0 (1 step):", P0_european_1step)
print("American call premium C0:", C0_american_1step)

Risk-neutral probability Up Move: 0.6000000000000001
Risk-neutral probability Down Move: 0.3999999999999999
Stock prices at maturity Up Move: 62.5
Payoff Up Move 17.5
Stock prices at maturity Down Move:: 40.0
Payoff Up Move: 0.0
Continuation value (European premium): 9.813084112149534
Intrinsic value today: 5.0
European put premium P0 (1 step): 9.813084112149534
American call premium C0: 9.813084112149534


### 2 steps

In [18]:
# Each step is now 0.5 years (since total T = 1 year, N = 2)
T  = 1.0          # total time to maturity in years
N  = 2            # number of steps
dt = T / N        # length of each step = 1.0 / 2 = 0.5 years

# Per-step gross risk-free return
R2 = (1 + Rf) ** dt # = (1 + 0.07)^0.5 ≈ 1.034

# `Risk-neutral probability on up move - 2 step`
prob_U2 = (R2 - D) / (U - D) # = (1.0344080433 - 0.80) / (1.25 - 0.80) ≈ 0.52

# `Risk-neutral probability on down move - 2 step`
prob_D2 = 1 - prob_U2  # 1 - 0.52 = 0.48

## (t = 0.5 year, after 1 step)
# Su = 62.5 = 50 * 1.25
# Sd = 40 = 50 * 0.8


# 1) STOCK PRICES AT MATURITY (t = 1 year, after 2 steps). Multiply by 2 values for two steps. I.E 50 * 1.25 = 62.5 * 1.25 = 78.13

 # = 50 * 1.25 * 1.25 = 78.125
Su2 = S0 * U * U   # up then up

# New Branch. Outcome of  Su (62.5) and Sd (40)
 # = 50 * 1.25 * 0.80 = 50.0
S_ud = S0 * U * D   # up then down (or down then up, same price in recomb. tree)

 # = 50 * 0.80 * 0.80 = 32.0
Sd2 = S0 * D * D   # down then down

# 2) PAYOFFS AT MATURITY (t = 1) FOR EACH NODE, find C in t = 1
Cu2 = max(Su2 - K, 0.0)    # = max(78.125 - 45, 0) = 33.125 
C_ud = max(S_ud - K, 0.0)  # = max(50.0  - 45, 0) = 5.0
Cd2 = max(Sd2 - K, 0.0)    # = max(32.0  - 45, 0) = 0.0


# 3) STEP BACK TO TIME 0.5 (t = 0.5)
#    First handle the node where stock went UP in step 1: Su = S0 * U
Su = S0 * U  # 50 * 1.25 = 62.5 

# continuation value at up-node = `discounted expected value` from Cu2 and C_ud, FOR EACH NODE, find C in t = 0.5
cont_up = ((prob_U2 * Cu2) + (prob_D2 * C_ud)) / R2 # = (0.5209067628 * 33.125 + 0.4790932372 * 5.0) / 1.0344080433 ≈ 18.997
intr_up = max(Su - K, 0.0)     # = max(62.5 - 45, 0) = 17.5 (intrinsic at that node)
# C = 18.997
C_up = max(intr_up, cont_up)       # = max(17.5, 18.99685799) ≈ 18.99685799

# For a EUROPEAN call, value at up node is just the continuation value:
C_up_eur = cont_up      # ≈ 18.9969
# (we do NOT compare to intrinsic here)

#    Now handle the DOWN node: S_down = S0 * D
S_down   = S0 * D
cont_down = ((prob_D2 * Cd2) + (prob_U2 * C_ud)) / R2 # = (0.4790932372 * 0 + 0.5209067628 * 5.0) / 1.0344080433 ≈ 2.517
intr_down = max(S_down - K, 0.0) # = max(40.0 - 45, 0) = 0.0
# C = 2.518
C_down = max(intr_down, cont_down)  # = max(0.0, 2.51789788) ≈ 2.51789788

# For a EUROPEAN call, value at down node is also just continuation:
C_down_eur = cont_down  # ≈ 2.5179

# 4) STEP BACK TO TODAY, find C in t = 0
# continuation value at t = 0 from the up and down nodes

#(prob_U2 * C_up)) + ((prob_D2 * C_down)) / R2
cont_0 = ((prob_U2 * C_up) + (prob_D2 * C_down)) / R2 # = (0.5209067628 * 18.99685799 + 0.4790932372 * 2.51789788) / 1.0344080433 ≈ 10.73261148

# continuation value at t = 0 from the European mid-node values
cont_0_eur = ((prob_U2 * C_up_eur) + (prob_D2 * C_down_eur)) / R2 # = (0.5209067628 * 18.99685799 + 0.4790932372 * 2.51789788) / 1.0344080433 ≈ 10.7326

intr_0 = max(S0 - K, 0.0)   # = max(50 - 45, 0) = 5.0 (intrinsic today, same as step 1)

C0_american_2step = max(intr_0, cont_0) # = max(5.0, 10.73261148) ≈ 10.73261148

print("\n=== 2-STEP Binomial CALL (European) ===")
print("Per-step R_step:", R2)
print("Per-step prob_U:", prob_U2)
print("Stock at uu, ud, dd:", Su2, S_ud, Sd2)
print("Payoffs C_uu, C_ud, Cd2:", Cu2, C_ud, Cd2)
print("Value at up node (t=0.5):", C_up_eur)
print("Value at down node (t=0.5):", C_down_eur)
print("Continuation at t=0:", cont_0)
print("Intrinsic at t=0:", intr_0)
print("American call premium (2 step):", C0_american_2step)
print("European call premium (2 step):", cont_0_eur)


=== 2-STEP Binomial CALL (European) ===
Per-step R_step: 1.03440804327886
Per-step prob_U: 0.5209067628419112
Stock at uu, ud, dd: 78.125 50.0 32.0
Payoffs C_uu, C_ud, Cd2: 33.125 5.0 0.0
Value at up node (t=0.5): 18.996857992945138
Value at down node (t=0.5): 2.517897875149657
Continuation at t=0: 10.732611484652352
Intrinsic at t=0: 5.0
American call premium (2 step): 10.732611484652352
European call premium (2 step): 10.732611484652352


#### N steps Model

In [20]:
S0 = 50.0
K  = 70.0         
sigma = 1
Rf = 0.10          
T  = 1.0           
N  = 100           
dt = T / N

# Per-step risk-free growth factor (simple compounding per step)
R = 1 + Rf * dt    # = 1 + 0.10 * (1/50)

# Up and down factors from volatility
U = math.exp(sigma * math.sqrt(dt))
D = 1 / U

# Risk-neutral probabilities
prob_U = (R - D) / (U - D)
prob_D = 1 - prob_U


# Loop Logic
# I. Payoffs at maturity (T) 
C = []  # option values at terminal nodes
for i in range(N + 1):
    # price at maturity: S_T = S0 * U^(N-i) * D^i
    ST = S0 * (U**(N - i)) * (D**i)
    C.append(max(ST - K, 0.0))   # CALL payoff: max(S_T - K, 0)


# II. Backward induction
# American trees calculated 
for step in range(N, 0, -1):
    newC = []
    for i in range(step):
        # continuation value (risk-neutral discounted expectation)
        continuation = (prob_U * C[i] + prob_D * C[i + 1]) / R

        # underlying stock price at this node (time = (step-1)*dt)
        S_node = S0 * (U**(step - 1 - i)) * (D**i)

        # intrinsic value if exercised now (CALL)
        intrinsic = max(S_node - K, 0.0)

        # American call value at this node
        american_val = max(continuation, intrinsic)

        newC.append(american_val)

    C = newC[:]  # move to previous time layer

C0_american_50step = C[0]


# III. European value calculated separately (without early exercise) 
# rebuild payoff structure & do backward induction again
C_eur = []
for i in range(N + 1):
    ST = S0 * (U**(N - i)) * (D**i)
    C_eur.append(max(ST - K, 0.0))

for step in range(N, 0, -1):
    C_eur = [
        (prob_U * C_eur[i] + prob_D * C_eur[i + 1]) / R
        for i in range(step)
    ]

C0_european_50step = C_eur[0]

# print results
print(f"European Call ({N} step) C0:", C0_european_50step)
print(f"American Call ({N} step) C0:", C0_american_50step)



European Call (100 step) C0: 15.61055281181013
American Call (100 step) C0: 15.61055281181013


## Call - Loop Logic

In [22]:
## N periods - 1, 2, 10, 50, 100
## sigma - 0.2, 0.5, 1.0
def binomial_call(S0, K, sigma, Rf, T, N):
    dt = T / N
    R  = 1 + Rf * dt             
    U  = math.exp(sigma * math.sqrt(dt))
    D  = 1 / U

    prob_U = (R - D) / (U - D)
    prob_D = 1 - prob_U

    # I. Payoffs at maturity (T)
    C_amer = []
    for i in range(N + 1):
        ST = S0 * (U**(N - i)) * (D**i)
        C_amer.append(max(ST - K, 0.0))   # call payoff

    C_eur = C_amer.copy()

    # II. Backward induction
    for step in range(N, 0, -1):
        # European: continuation only
        C_eur = [
            (prob_U * C_eur[i] + prob_D * C_eur[i + 1]) / R
            for i in range(step)
        ]

        # American: max(continuation, intrinsic)
        newC = []
        for i in range(step):
            continuation = (prob_U * C_amer[i] + prob_D * C_amer[i + 1]) / R
            S_node = S0 * (U**(step - 1 - i)) * (D**i)
            intrinsic = max(S_node - K, 0.0)
            newC.append(max(continuation, intrinsic))
        C_amer = newC

    return C_eur[0], C_amer[0]

In [23]:
## N periods - 1, 2, 10, 50, 100
## sigma - 0.2
S0    = 50.0
K     = 70.0
sigma = 0.2       
Rf    = 0.10
T     = 1.0

steps_list = [1, 2, 10, 50, 100]

print("Steps   Eur_Call      Amer_Call")
for N in steps_list:
    C_eur, C_amer = binomial_call(S0, K, sigma, Rf, T, N)
    print(f"{N:<6} {C_eur:<12.6f} {C_amer:<12.6f}")


Steps   Eur_Call      Amer_Call
1      0.000000     0.000000    
2      0.000000     0.000000    
10     0.603549     0.603549    
50     0.625825     0.625825    
100    0.647409     0.647409    


In [24]:
## N periods - 1, 2, 10, 50, 100
## sigma - 0.5
S0    = 50.0
K     = 70.0
sigma = 0.5       
Rf    = 0.10
T     = 1.0

steps_list = [1, 2, 10, 50, 100]

print("Steps   Eur_Call      Amer_Call")
for N in steps_list:
    C_eur, C_amer = binomial_call(S0, K, sigma, Rf, T, N)
    print(f"{N:<6} {C_eur:<12.6f} {C_amer:<12.6f}")

Steps   Eur_Call      Amer_Call
1      5.353074     5.353074    
2      6.611921     6.611921    
10     5.495341     5.495341    
50     5.739956     5.739956    
100    5.726291     5.726291    


In [25]:
## N periods - 1, 2, 10, 50, 100
## sigma - 1
S0    = 50.0
K     = 70.0
sigma = 1       
Rf    = 0.10
T     = 1.0

steps_list = [1, 2, 10, 50, 100]

print("Steps   Eur_Call      Amer_Call")
for N in steps_list:
    C_eur, C_amer = binomial_call(S0, K, sigma, Rf, T, N)
    print(f"{N:<6} {C_eur:<12.6f} {C_amer:<12.6f}")

Steps   Eur_Call      Amer_Call
1      18.664913    18.664913   
2      16.197261    16.197261   
10     15.987916    15.987916   
50     15.587069    15.587069   
100    15.610553    15.610553   
